In [1]:
import pandas as pd

from convertbng.util import convert_bng, convert_lonlat
import numpy as np

pd.set_option('max_colwidth',None)

DIR ='./Data/'

## Column Filtering

### Cleaning Accidents Dataset

In [6]:
# reading in required columns
acc_col_list = ['accident_index', 'location_easting_osgr', 'location_northing_osgr','latitude','longitude', 'number_of_vehicles',
       'number_of_casualties', 'date', 'day_of_week', 'time',]
acc_df = pd.read_csv(DIR + '/raw/accidents_raw.csv', usecols = acc_col_list)

In [8]:
# filtering data from 2000
acc_df['date'] = pd.to_datetime(acc_df['date'], infer_datetime_format=True)
acc_2000 = acc_df[acc_df['date'] > '31-12-1999']
del acc_df

In [9]:
# filter not null easting/northing 
acc_2000 = acc_2000[acc_2000['location_easting_osgr'].notna() & acc_2000['location_northing_osgr'].notna()]

In [10]:
# convert easting and northing to lat long for missing rows
mask = acc_2000['latitude'].isna()

long_lat = convert_lonlat(acc_2000.loc[mask,['location_easting_osgr']].values, 
                        acc_2000.loc[mask,['location_northing_osgr']].values)
                                               
acc_2000.loc[mask,['longitude','latitude']] = np.array(long_lat).T

In [12]:
# Filtering out 
acc_2000 = acc_2000[acc_2000['longitude'].notna() & acc_2000['latitude'].notna()]

In [14]:
acc_2000.to_csv(DIR+'acc_clean.csv', index=False)

### Filtering Vehicles Dataset

In [5]:
v_cols_list = ['accident_index', 'vehicle_reference', 'vehicle_type']
vehicles_df = pd.read_csv(DIR+'/raw/vehicles_raw.csv',usecols = v_cols_list)
vehicles_df.to_csv(DIR+'/vehicles_clean.csv',index=False)
del vehicles_df

d:\installations\anaconda3\envs\ke\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Filtering and Cleaning Pubs Dataset

In [2]:
pubs_cols = ['fas_id','name','latitude','longitude','local_authority']
pubs_df = pd.read_csv(DIR+'raw/open_pubs_raw.csv',usecols=pubs_cols)

In [3]:
# removing rows with invalid coordinates
mask = ~pubs_df.latitude.str.contains('N')
pubs_df = pubs_df[mask]
string_filter = ~ (pubs_df.longitude.str.contains('[A-Z]+[a-z]+') | pubs_df.longitude.str.contains('[A-Z]+[a-z]+'))
pubs_df = pubs_df[string_filter]

In [4]:
pubs_df.to_csv(DIR+'pubs_clean.csv', index = False)
del pubs_df